## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_to_load = os.path.join('..','new_weather_database','WeatherPy_Database.csv')

city_data_df = pd.read_csv(city_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,69.26,59,75,3.00,broken clouds
1,1,Qaanaaq,GL,77.48,-69.36,20.95,84,100,5.32,overcast clouds
2,2,Port Elizabeth,ZA,-33.92,25.57,64.09,100,40,12.66,scattered clouds
3,3,Avarua,CK,-21.21,-159.78,82.45,78,20,6.91,light rain
4,4,Thompson,CA,55.74,-97.86,-20.04,76,1,8.05,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

select_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp) &
                                      (city_data_df['Max Temp']>=min_temp)]


In [5]:
# 4a. Determine if there are any empty rows.
select_cities_df.count()

City_ID                171
City                   171
Country                171
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
select_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.21,-159.78,82.45,78,20,6.91,light rain
6,6,Vaini,TO,-21.20,-175.20,84.36,84,20,12.66,few clouds
14,14,Rikitea,PF,-23.12,-134.97,76.69,73,87,6.26,overcast clouds
30,30,Kapaa,US,22.08,-159.32,79.12,67,40,3.00,light rain
40,40,Iquitos,PE,-3.75,-73.25,77.04,94,75,1.14,broken clouds
...,...,...,...,...,...,...,...,...,...,...
650,650,Cabangahan,PH,9.08,122.95,80.60,73,100,4.79,overcast clouds
651,651,Anloga,GH,5.79,0.90,82.62,81,67,11.92,broken clouds
652,652,Los Andes,CL,-32.83,-70.60,77.72,39,39,0.72,scattered clouds
654,654,Lazaro Cardenas,MX,17.96,-102.20,77.97,83,86,4.81,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = select_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,82.45,light rain,-21.21,-159.78,
6,Vaini,TO,84.36,few clouds,-21.20,-175.20,
14,Rikitea,PF,76.69,overcast clouds,-23.12,-134.97,
30,Kapaa,US,79.12,light rain,22.08,-159.32,
40,Iquitos,PE,77.04,broken clouds,-3.75,-73.25,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
     
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"




    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    search_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(search_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found.....skipping")
        
        

Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,82.45,light rain,-21.21,-159.78,Paradise Inn
6,Vaini,TO,84.36,few clouds,-21.20,-175.20,Keleti Beach Resort
14,Rikitea,PF,76.69,overcast clouds,-23.12,-134.97,People ThankYou
30,Kapaa,US,79.12,light rain,22.08,-159.32,Sheraton Kauai Coconut Beach Resort
40,Iquitos,PE,77.04,broken clouds,-3.75,-73.25,Cortez Malaga Techy Caroly
...,...,...,...,...,...,...,...
650,Cabangahan,PH,80.60,overcast clouds,9.08,122.95,Centralle District Beach Resort
651,Anloga,GH,82.62,broken clouds,5.79,0.90,Pin Drop Hotel
652,Los Andes,CL,77.72,scattered clouds,-32.83,-70.60,Hotel Los Andes
654,Lazaro Cardenas,MX,77.97,overcast clouds,17.96,-102.20,quinta antigua


In [11]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join('..','vacation_search','WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
# 11b. Display the figure

fig = gmaps.figure(center=(30.0,31.0),zoom_level = 1.7)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))